# Stress Testing — Q3 2024

Applies macro stress scenarios to baseline loss forecast.

Data source: `portfolio_risk.db`
Model reference: `loss_forecast_model.py`
Policy reference: `model_methodology.docx`

In [ ]:
import sqlite3
import pandas as pd
import numpy as np

DB_PATH = "portfolio_risk.db"
conn = sqlite3.connect(DB_PATH)

In [ ]:
# Load historical defaults
defaults = pd.read_sql(
    "SELECT * FROM historical_defaults ORDER BY quarter DESC", conn
)

# Use 4-quarter average for baseline (faster iteration during stress runs)
baseline_rate = defaults.head(4)["default_rate"].mean()
print(f"Baseline rate (4Q avg): {baseline_rate:.4f}")

Baseline rate (4Q avg): 0.0585


In [ ]:
# Macro stress scenarios
# NOTE: Overlay factors are Alice's expert estimates — not from any formal model
#       These have never been peer-reviewed or validated.
STRESS_SCENARIOS = {
    "base":                {"gdp_shock": 0.0,  "unemp_shock": 0.0, "overlay": 0.000},
    "mild_downturn":       {"gdp_shock": -0.5, "unemp_shock": 0.3, "overlay": 0.015},
    "moderate_recession":  {"gdp_shock": -1.5, "unemp_shock": 1.0, "overlay": 0.035},
    "severe_recession":    {"gdp_shock": -3.0, "unemp_shock": 2.5, "overlay": 0.050},
}

In [ ]:
# Run stress scenarios
results = []
for name, params in STRESS_SCENARIOS.items():
    adjusted_rate = baseline_rate + params["overlay"]
    results.append({
        "scenario": name,
        "baseline": round(baseline_rate, 4),
        "overlay": params["overlay"],
        "stressed_rate": round(adjusted_rate, 4),
    })

stress_df = pd.DataFrame(results)
print(stress_df.to_string(index=False))

           scenario  baseline  overlay  stressed_rate
               base    0.0585    0.000         0.0585
      mild_downturn    0.0585    0.015         0.0735
  moderate_recession    0.0585    0.035         0.0935
   severe_recession    0.0585    0.050         0.1085


In [ ]:
# Compare with actual forecast (Alice's manual overlays)
forecast_df = pd.read_excel("Q3_2024_forecast.xlsx")
print(forecast_df)

## Notes

- Baseline uses **4-quarter average** for speed — policy says 12-quarter
  (see `risk_queries.sql` for both variants)
- Overlay factors in `STRESS_SCENARIOS` are Alice's expert judgment estimates
- **TODO:** Get overlay factors peer-reviewed before year-end
- The severe recession overlay (+5%) matches the overlay cap in `portfolio_risk.db`
  (`policy_snapshot.overlay_cap = 0.05`)